In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF

In [ ]:
class DoubleConv(nn.Module):
  def __init__(self,in_channel,out_channel):
      super(DoubleConv,self).__init__()
      
      self.conv=nn.Sequential(
          nn.Conv2d(in_channels=in_channel,out_channels=out_channel,kernel_size=3,stride=1,padding=1,bias=False),
          nn.BatchNorm2d(out_channel),
          nn.ReLU(inplace=True),

          nn.Conv2d(in_channels=out_channel,out_channels=out_channel,kernel_size=3,stride=1,padding=1,bias=False),
          nn.BatchNorm2d(out_channel),
          nn.ReLU(inplace=True),
          )

  def forward(self,x):
      return self.conv(x)
# class Down(nn.Module):
#   def __init__(self,in_channel,out_channel,mid_channel=None):
#     super().__init__()
#     self.doubleConv=DoubleConv(in_channel=in_channel,out_channel=out_channel,mid_channel=mid_channel)
#     self.maxPool=nn.MaxPool2d(kernel_size=2,stride=2)

#   def forward(self,x):
#       x=self.doubleConv(x)
#       return self.MaxPool(x)

# class Up(nn.Module):
#   def __init__(self,in_channel,out_channel):
#     super().__init__()
#     self.deConv=nn.ConvTranspose2d(in_channels=in_channel,out_channels=in_channel//2,kernel_size=2,stride=2)
    # self.doubleConv=DoubleConv(in_channel=in_channel,out_channel=out_channel)

  # def forward(self,x,x_skip):
  #     x=self.deConv(x)
  #     x=torch.cat((x,x_skip),dim=1)
  #     return self.doubleConv(x)

class Unet(nn.Module):
  def __init__(self,in_channels=3,out_channels=1,features=[64,128,256,512]):
    super(Unet,self).__init__()
    self.ups=nn.ModuleList()
    self.downs=nn.ModuleList()
    self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
    #self.inc=DoubleConv(in_channel,64)
    #self.down1=Down(64,128)
    #self.down2=Down(128,256)
    #self.down3=Down(256,512)
    #self.down4=Down(512,1024)

    #self.up1=Up(1024,512)
    #self.up2=Up(512,256)
    #self.up3=Up(256,128)
    #self.up4=Up(128,64)

    #self.conv=nn.Conv2d(64,num_class,kernel_size=1)

    #Down part of Unet

    for feature in features:
      self.downs.append(DoubleConv(in_channels,feature))
      in_channels=feature
    

    #Ups part of Unet
    for feature in reversed(features):
        self.ups.append(
            nn.ConvTranspose2d(feature*2,feature,kernel_size=2,stride=2))
        self.ups.append(DoubleConv(feature*2,feature))

    self.bottleneck=DoubleConv(features[-1],features[-1]*2)
    self.final_conv=nn.Conv2d(features[0],out_channels,kernel_size=1)

  def forward(self,x):
    skip_connections=[]

    for down in self.downs:
      x=down(x)
      skip_connections.append(x)
      x=self.pool(x)
    x=self.bottleneck(x)
    skip_connections=skip_connections[::-1]
    for idx in range(0,len(self.ups),2):
      x=self.ups[idx](x)
      skip_connection=skip_connections[idx//2]
      if x.shape!=skip_connection.shape:
        x=TF.resize(x,size=skip_connection.shape[2:])
        concat_skip=torch.cat((skip_connection,x),dim=1)
        x=self.ups[idx+1](concat_skip)
    return self.final_conv(x)
    # inc=self.inc(x)
    # d1=self.down1(inc)
    # d2=self.down2(d1)
    # d3=self.down3(d2)
    # d4=self.down4(d3)

    # x=self.up1(d4,d3)
    # x=self.up2(x,d2)
    # x=self.up3(x,d1)
    # x=self.up4(x,inc)
    # x=self.conv(x)
    # return x

def test():
   x=torch.randn((3,1,161,161)) 
   model=Unet(in_channels=1,out_channels=1)
   preds=model(x)
   assert preds.shape==x.shape

if __name__== "__main__":
  test()

# if __name__=="__main__":
#   in_img=torch.randn(1,3,640,640)
#   model=Unet(3,5)
#   prediction=model(in_img)
#   print(prediction.size())

